In [ ]:
%run Weighting.ipynb

In [ ]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

In [ ]:
nilai_jams = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]

In [ ]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)
params = get_params(mean, nilai_jams, std)

params

In [ ]:
tabel_alternatif = add_parameter(params, tabel_alternatif)

tabel_alternatif

In [ ]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

for pilihan in pilihan_jurusan:
    if "(ipa)" in pilihan.casefold().split():
        print(pilihan)

In [ ]:
fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

fp_gaussian

In [ ]:
index_preferensi_global = idx_pref_global(fp_gaussian, weight)

index_preferensi_global

In [ ]:
e_flow = entering_flow(pilihan_jurusan, index_preferensi_global)

e_flow

In [ ]:
l_flow = leaving_flow(pilihan_jurusan, index_preferensi_global)

l_flow

In [ ]:
n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

n_flow

In [ ]:
hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

tab = dict()

rank = list()
jurusan = list()
net = list()

for i in range(len(hasil_outrank)):
    rank.append(i+1)
    jurusan.append(hasil_outrank[i][0])
    net.append(hasil_outrank[i][1])
    
tab["Jurusan"] = jurusan
tab["Net Flow"] = net

tabel_hasil = pd.DataFrame(tab, index=rank)

tabel_hasil